In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import xml.etree.ElementTree as ET
import libfmp.b

In [ ]:
with open('model/songs_with_errors.txt') as f:
    songs = f.readlines()

In [ ]:
songs = list(map(lambda x:x.replace('\n',''), songs))
songs

Investiguei as músicas com erros para tentar identificar suas principais razões. Pelo que pude observar, são dois os principais motivos para isso: (i) músicas que têm duas vozes em vez de apenas uma linha com a melodia. Nesses casos, a linha em que se encontra a harmonia varia, confundindo nosso código original;
(ii) muitos compassos de pausa seguidos. O algoritmo do music21, que utilizamos para gerar os audios a partir dos arquivos xml, os ignora, ocasionando divergências do áudio com a sequência de acordes.

Vou elencar os tipos de erro para cada música:

Aqui vamos gerar as sequências de acordes explícitas do nosso corpus de audio.

In [ ]:
songs = os.listdir('corpus/xml')
for song in songs:
    tree = ET.parse('corpus/xml/{}'.format(song))
    root = tree.getroot()
    song_title = song.split('.')[0]
    df_harmony = pd.DataFrame(columns=[song_title,'chord_note'])

    part = root.findall('part')[0]
    measures = part.findall('measure')

    for measure in measures:
        chords = measure.findall('harmony')
        for chord in chords:
            try:
                note = chord.find('root/root-step').text

            except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
                print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
                continue

            if note:
                # alter é a alteração de um acorde (nem sempre existe): b ou #.
                alter = chord.find('root/root-alter')#.text

                # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
                kind = chord.find('kind')#.attrib['text']

                # dissonancia adicional. se temos um acorde 7(9), a 7a aparece em 'kind' e a 9a em 'degree'
                degree = chord.find('degree')

                # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
                bass = chord.find('bass/bass-step')#.text

                # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
                bass_alter = chord.find('bass/bass-alter')#.text

                attribs_dict = {'alter':alter, 
                                'kind':kind, 
                                'degree':degree,
                                'bass':bass, 
                                'bass_alter':bass_alter}

                for key, value in attribs_dict.items():
                    if value is None:
                        attribs_dict[key] = ''
                    elif key == 'degree':
                        degree_value = attribs_dict[key].find('degree-value').text
                        degree_alter = attribs_dict[key].find('degree-alter').text

                        attribs_dict[key] = '(' + degree_alter + degree_value + ')'

                    elif key == 'kind':
                        try:
                            attribs_dict[key] = value.attrib['text']
                        except KeyError:
                            attribs_dict[key] = value.text#chord.find('kind').text
                    else:
                        attribs_dict[key] = value.text



                # formatando as alterações de acorde
                attribs_dict['alter'] = attribs_dict['alter'].replace('-1', 'b').replace('1', '#')
                attribs_dict['degree'] = attribs_dict['degree'].replace('-1', 'b').replace('1', '#')
                attribs_dict['bass_alter'] = attribs_dict['bass_alter'].replace('-1', 'b').replace('1', '#')
                
                # 11a dá problema, rs
                attribs_dict['alter'] = attribs_dict['alter'].replace('###', '#11').replace('##b','b11')
                attribs_dict['degree'] = attribs_dict['degree'].replace('###', '#11').replace('##b','b11')
                attribs_dict['bass_alter'] = attribs_dict['bass_alter'].replace('###', '#11').replace('##b','b11')
                

                measure_number = measure.attrib['number']
                chord = note + attribs_dict['alter'] + attribs_dict['kind']
                chord_bass = attribs_dict['bass'] + attribs_dict['bass_alter']
                degree = attribs_dict['degree']

                complete_chord = chord + degree + '/' + chord_bass

                if complete_chord.endswith('/'):
                    complete_chord = complete_chord[:-1]

                df_chord = pd.DataFrame(columns=[song_title,'chord_note'], 
                                        index=[measure_number])
                
                df_chord[song_title] = complete_chord
                df_chord['chord_note'] = note + attribs_dict['alter']
                
                df_harmony = df_harmony.append(df_chord)

    df_harmony.index.names = ['Measure']



    # tipo de compasso
    first_measure = measures[0]
    cima = first_measure.find('attributes').find('time/beats').text
    baixo = first_measure.find('attributes').find('time/beat-type').text

    compasso = cima + '/' + baixo

    # andamento
    andamento = 120

    # sync: acorde + minutagem
    duracao_seminima = 60/int(andamento)
    segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima
    df_harmony.index = df_harmony.index.astype(int)

    for compasso in df_harmony.index.unique():
        acordes = df_harmony.loc[[compasso]]
        n_acordes = len(acordes)
    #     print(compasso, n_acordes)
        inicio_compasso = (int(compasso) - 1) * segs_compasso

        if n_acordes == 1:
            tempos = inicio_compasso

        else:        
            tempos = np.linspace(inicio_compasso,
                                 inicio_compasso + segs_compasso,
                                 n_acordes+1)
            tempos = tempos[:-1]

        df_harmony.at[compasso, 'Time'] = tempos


        df_harmony.to_csv('corpus/chords_sequences/{}.csv'.format(song_title))
    print(song_title)